## This notebook creates the tectonic inputs of the testing cases. 

### In all testing cases, the only tectonic forcing is the migrating dynamic topography wave.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import cm
import matplotlib.mlab as ml
import matplotlib.pyplot as plt

%matplotlib inline

## Creating a dynamic topography wave 

$$ z(x) = z_0 e^{-(x-\nu t)^2/\lambda^2} $$

- $\nu$ wave velocity
- $2\lambda$ wavelength
- $t$ time
- $z_0$ maximum amplitude

In [ ]:
# Parameter of dynamic topography (DT)
v = 0.05  # velocity of DT (m/yr)
l = 500000.  # wavelength (m)
zo = 300.  # amplitude (m)
# 
xr_DT = np.linspace(-500000., 500000., 1001)
l1 = 400000.
DT1 = zo*np.exp(-np.pi*(xr_DT)*(xr_DT)/l1/l1)
DT = DT1 - DT1[0]
plt.plot(xr_DT,DT1,xr_DT,DT)

### Model geometry

In [ ]:
# Number of nodes along X-Y axis
nx = 701  
ny = 701  
xr = np.linspace(0., 700000., nx)
yr = np.linspace(0., 700000., ny)

# Space resolution along X-Y axis
dx = xr[1] - xr[0]  
dy = yr[1] - yr[0]  

# Time array
endTime = 34000000 #  Dynamic topogrpahy passes the domain in 34 My
dispTimeStep = dx*5/v  # every DT move travels 5 dx
stepNb = int(endTime/dispTimeStep)
dispTimeStep, stepNb

### Export data

In [ ]:
z = np.zeros((ny,nx))
for k in range(0,stepNb):
    f = open("disp"+str(k)+".csv", "w")
    t = (k+1)*dispTimeStep

    disp = np.zeros((ny,nx))
    zn = np.zeros((ny,nx))
    beta = np.zeros((nx,ny))
    r = np.zeros((nx,ny))

    posit = int(t*v/dx)
    lenDT = int(DT.shape[0]-1)
    if (posit<=700):
        for j in range(ny):
            for i in range(0,posit,1):
                zn[j,i] = DT[(lenDT-posit+1)+i]
                disp[j,i] = zn[j,i] - z[j,i]
    elif (posit<=int(nx)):
        posit_pass = posit - int(l/dx)
        for j in range(ny):
            for i in range(posit_pass,posit,1):
                zn[j,i] = DT[i-posit_pass]
                disp[j,i] = zn[j,i] - z[j,i]
    else:
        posit_pass = posit - 700
        for j in range(ny):
            for i in range(posit_pass,nx,1):
                zn[j,i] = DT[i-posit_pass]
                disp[j,i] = zn[j,i] - z[j,i]
    z = np.copy(zn)

    for j in range(0,ny):
        for i in range(0,nx):
            f.write( str(disp[j][i]) + "\n"  )
    f.close()

### XML input

In [ ]:
endTime = 34000000
dispTimeStep = 100000
stepNb = int(endTime/dispTimeStep)
for k in range(0,stepNb):
    print("<disp>")
    print("   <dstart>"+str(k*dispTimeStep)+"</dstart>")
    print("   <dend>"+str((k+1)*dispTimeStep)+"</dend>")
    print("   <dfile>data/dtp300_1000k_v5_ts0/disp"+str(k)+".csv</dfile>")
    print("</disp>")